<a href="https://colab.research.google.com/github/poudel200/AI-lab/blob/main/bidirectional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def train_bam(id_vectors, phone_vectors):
    """
    Train the BAM using Hebbian learning.

    :param id_vectors: Binary vectors for ID
    :param phone_vectors: Binary vectors for phone numbers
    :return: BAM weights matrix
    """
    # Ensure that both id_vectors and phone_vectors have the same number of samples
    if id_vectors.shape[0] != phone_vectors.shape[0]:
        raise ValueError("ID vectors and Phone number vectors must have the same number of samples")

    # Initialize weights to zero
    num_id_features = id_vectors.shape[1]
    num_phone_features = phone_vectors.shape[1]

    # Create a BAM weight matrix (id -> phone, phone -> id)
    weights = np.zeros((num_id_features + num_phone_features, num_id_features + num_phone_features))

    # Train using Hebbian learning rule: outer product of input and output vectors
    for i in range(id_vectors.shape[0]):
        id_vector = id_vectors[i]
        phone_vector = phone_vectors[i]

        # Outer product for BAM weights (id -> phone and vice versa)
        input_vector = np.concatenate((id_vector, phone_vector))  # Concatenate ID and Phone vectors
        outer_product = np.outer(input_vector, input_vector)

        # Update the BAM weights
        weights += outer_product

    return weights

def bam_retrieve(weights, input_vector, vector_type='id'):
    """
    Retrieve the corresponding vector for given input using BAM.

    :param weights: Trained BAM weights
    :param input_vector: Input vector (either ID or phone number)
    :param vector_type: Type of input ('id' or 'phone')
    :return: Retrieved vector (phone number or ID)
    """
    # Construct the full input vector depending on the type (ID or Phone)
    if vector_type == 'id':
        full_input = np.concatenate((input_vector, np.zeros(weights.shape[0] // 2)))  # ID -> phone
    elif vector_type == 'phone':
        full_input = np.concatenate((np.zeros(weights.shape[0] // 2), input_vector))  # Phone -> ID
    else:
        raise ValueError("Input type should be 'id' or 'phone'")

    # Retrieve output vector by multiplying input with weights
    output_vector = np.sign(np.dot(weights, full_input))
    return output_vector

# Example Training Data
# 3 IDs with their respective phone numbers (binary representation)
id_vectors = np.array([
    [1, 0, 0],  # ID 1
    [0, 1, 0],  # ID 2
    [0, 0, 1]   # ID 3
])

phone_vectors = np.array([
    [1, 1, 0],  # Phone for ID 1
    [0, 1, 1],  # Phone for ID 2
    [1, 0, 1]   # Phone for ID 3
])

# Train BAM
weights = train_bam(id_vectors, phone_vectors)

# Retrieve phone number from ID
id_input = np.array([1, 0, 0])  # ID 1
retrieved_phone = bam_retrieve(weights, id_input, vector_type='id')
print("Retrieved Phone Number for ID 1:", retrieved_phone)

# Retrieve ID from phone number
phone_input = np.array([1, 1, 0])  # Phone for ID 1
retrieved_id = bam_retrieve(weights, phone_input, vector_type='phone')
print("Retrieved ID for Phone (1, 1, 0):", retrieved_id)
